In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_pickle('raw_data/final_data_set.pkl')

In [3]:
df.head()

,Product Label,X
0,Bag,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,Bag,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,Bag,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
3,Bag,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,Bag,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."


In [4]:
df_2 = df.reset_index()

In [5]:
df_2['X'][0].shape

(256, 256, 3)

In [7]:
list1 = []
list_y=[]
missing_idX=[]
for i, v in enumerate(df_2['X'][0:14999]):
    #if len(v.shape)==2:
        #missing_idX.append(i)
    if v.shape==(256,256,3):
        list1.append(v)
    else:
        missing_idX.append(i)
    if i not in missing_idX:
        list_y.append(df_2['Product Label'][i])

In [8]:
len(list_y)

9993

In [9]:
len(missing_idX)

5006

In [10]:
len(list1)

9993

In [11]:
np.array(list1).shape

(9993, 256, 256, 3)

In [12]:
X = np.array(list1)

In [13]:
y = list_y

In [36]:
np.unique(y)

array(['Bag', 'Bed Linen', 'Blanket', 'Box', 'Bracelet', 'Candle',
       'Coaster', 'Cushion Cover', 'Dress', 'Earring', 'Face Care',
       'Food', 'Gift Box'], dtype='<U13')

In [14]:
import pandas as pd
#from matplotlib import pyplot as plt
import os, json
import pandas as pd
from PIL import Image
import numpy as np
from numpy import asarray
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from google.cloud import storage

In [15]:
encoder = LabelEncoder()


In [16]:
y_encod = encoder.fit_transform(y)


In [32]:
y_encod

array([ 0,  0,  0, ..., 12, 12, 12])

In [17]:
y_encod.shape


(9993,)

In [33]:
np.unique(y_encod)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [19]:
y_cat = to_categorical(y_encod, num_classes=len(np.unique(y_encod)))


In [20]:
y_cat.shape

(9993, 13)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.3)

In [22]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6995, 256, 256, 3) (2998, 256, 256, 3) (6995, 13) (2998, 13)


In [23]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import pandas as pd
import numpy as np
import joblib
from google.cloud import storage

In [24]:
model = Sequential()
model.add(Rescaling(1./255, input_shape=(256, 256, 3)))

model.add(layers.Conv2D(16, (10,10), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(3,3)))

model.add(layers.Conv2D(32, (8,8), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(3,3)))

model.add(layers.Conv2D(32, (6,6), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(3,3)))

model.add(layers.Flatten())
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(13, activation='softmax'))

In [25]:
learning_rate=1e-4
opt = optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])

In [26]:
es = EarlyStopping(patience = 20, restore_best_weights = True)
model.fit(X_train, y_train,
        batch_size=16,
        epochs=2,
        validation_split=0.3,
        callbacks=[es],
        verbose = 1)

Epoch 1/2
306/306 [==============================] - 714s 2s/step - loss: 1.7568 - accuracy: 0.4569 - val_loss: 1.5206 - val_accuracy: 0.5298
Epoch 2/2
306/306 [==============================] - 670s 2s/step - loss: 1.4279 - accuracy: 0.5623 - val_loss: 1.3752 - val_accuracy: 0.5612


In [28]:
score = model.evaluate(X_test, y_test, verbose=2)

94/94 - 55s - loss: 1.3250 - accuracy: 0.5827 - 55s/epoch - 589ms/step


In [29]:
joblib.dump(model, 'model.joblib')

2022-07-15 17:52:41.121288: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://7626b903-1e46-4668-845c-ad503c9119ea/assets


['model.joblib']

In [30]:
pwd

'/Users/Safaemichelot/code/aidakd/matjari'

In [31]:
ls

Image_matjari_test.png   dist/                    prediction_test.ipynb
MANIFEST.in              matjari/                 raw_data/
Makefile                 matjari.egg-info/        requirements.txt
README.md                model.joblib             scripts/
Untitled.ipynb           models_model_cnn.joblib  setup.py
Untitled2.ipynb          notebooks/               tests/
Untitled3.ipynb          pickl test.ipynb
